In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['LD_LIBRARY_PATH'] = "$LD_LIBRARY_PATH:/usr/local/cuda-11.3/lib64/"
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork, Trajectory
#import torch
#import torch.nn as nn
#from torch.utils.data import DataLoader
#import pandas as pd
import networkx as nx
import osmnx as ox
from models.hrnr_original.utils import *
from models.hrnr_original.conf import beijing_hparams
# from models.hrnr_original.train import train_struct_cmt, train_fnc_cmt_rst
from models.hrnr_original.model import *
import torch
import torch.nn.functional as F
from torch import optim
from models.training.hrnr_data_generation import *
from models import HRNRModel

In [2]:
torch.cuda.set_device(0)

In [3]:
# # try on original
# hparams = dict_to_object(beijing_hparams)
# hparams.device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# adj, features, sp_label = load_gae_data(hparams) 

In [4]:
# print(len(np.unique(features[:, 2])))

In [5]:
# train_struct_cmt()

In [6]:
# train_fnc_cmt_rst()

In [3]:
city = "hanover"
city_traj = "hanover"

In [4]:
# load road network
network = RoadNetwork()
network.load(f"../../osm_data/{city}")
data = network.generate_road_segment_pyg_dataset(dataset=city)


In [5]:
len(network.gdf_edges["lanes"].unique())

21

In [5]:
trajectory = Trajectory(
    f"../../datasets/trajectories/{city_traj}/road_segment_map_final.csv", nrows=10000000
)

Pandas Apply:   0%|          | 0/42357 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/42357 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/42357 [00:00<?, ?it/s]

In [6]:
adjp, featuresp, sp_labelp, t_adjp = get_data(network, trajectory, remove_highway_label=True)

100%|██████████| 42357/42357 [00:06<00:00, 6736.01it/s]


In [7]:
# train structural region and functional region matrices
train_struct_cmt_custom(adjp, featuresp, sp_labelp, city=city)
train_fnc_cmt_rst_custom(adjp, featuresp, sp_labelp, t_adjp, city=city)


49.9288215637207
42.924537658691406
44.18714141845703
43.1088752746582
38.57252883911133
40.676551818847656
27.877742767333984
38.454254150390625
27.40487289428711
50.0
37.1309700012207
37.04374313354492
32.00189208984375
30.32632827758789
50.0
50.0
50.0
50.0
28.962772369384766
29.25288963317871
18.18088150024414
12.679256439208984
9.295267105102539
7.393372058868408
6.9486284255981445
5.8768510818481445
4.956083297729492
4.050401210784912
4.126492023468018
4.442972660064697
3.6665432453155518
4.0371527671813965
3.2874577045440674
2.958665132522583
2.8865041732788086
2.6522650718688965
2.9837591648101807
3.5465900897979736
2.919703483581543
2.749927520751953
2.848558187484741
3.3113579750061035
2.3233940601348877
2.6851110458374023
2.9763848781585693
2.1552934646606445
2.482848644256592
2.1729822158813477
2.336660861968994
1.9852768182754517
1.949154257774353
2.3397951126098633
2.0603857040405273
2.2189040184020996
1.737504005432129
1.85929274559021
1.5216134786605835
1.874409675598144

In [11]:
# Build model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = HRNRModel(data, device=device, network=network, data_path="hrnr_data_noroad", remove_highway_label=True, city=city)

In [12]:
model.train()

Epoch: 50, avg_loss: 6.614514529705048
Epoch: 100, avg_loss: 3.7778184521198273
Epoch: 150, avg_loss: 2.8077241603533425
Epoch: 200, avg_loss: 2.31691821038723
Epoch: 250, avg_loss: 2.0191866788864137
Epoch: 300, avg_loss: 1.8188057444492975
Epoch: 350, avg_loss: 1.674555172579629
Epoch: 400, avg_loss: 1.5656696759164332
Epoch: 450, avg_loss: 1.480472988155153
Epoch: 500, avg_loss: 1.4119587584733964
Epoch: 550, avg_loss: 1.3556901824474334
Epoch: 600, avg_loss: 1.3085495495796204
Epoch: 650, avg_loss: 1.2684812160638663
Epoch: 700, avg_loss: 1.2340022147553307
Epoch: 750, avg_loss: 1.2040417215824126
Epoch: 800, avg_loss: 1.1777086959034204
Epoch: 850, avg_loss: 1.1543958042649662
Epoch: 900, avg_loss: 1.1336230714453592
Epoch: 950, avg_loss: 1.1149765209775222


In [13]:
model.save_model("../model_states/hrnr")

In [16]:
z = model.load_emb()

In [17]:
print(z.shape)

(11331, 128)
